# Image & Video Explorer
Pre-require: [ipywidgets](https://github.com/jupyter-widgets/ipywidgets), you can install it by executing `pip install ipywidgets` in terminal.   
Usage: Replace `ROOT` with directory that you want to explore, run the cell, then choose images from the `Dropdown` box.

In [4]:
# Change ROOT to the dictionary where you want to view
ROOT = "./example"
LOOP_TIME = 0.2  # Second for each image when AutoPlay
IMAGE_FMT = [
    '.jpg', '.jpeg', '.jpe', '.png', '.bmp', '.dib', '.tif', '.tiff', '.webp',
    '.pbm', '.pgm', '.ppm', '.sr', '.ras', '.exr', '.jp2'
]
VIDEO_FMT = ['.mp4', '.avi', '.mpg', '.mpeg', '.mov']

import os
import cv2
from time import sleep
from ipywidgets import Label, Text, Button, Dropdown, Image, HBox, VBox
from IPython.display import display

idx = 0
pics = []
info_lbl = Label()
pos_txt = Text(description="ROOT:")
pos_txt.layout.width = "90%"
jmp_btn = Button(description="Jump", button_style='primary')
jmp_btn.layout.width = "80px"
# idx_lbl = Label()
drpbox = Dropdown(description="[0/0]:")
drpbox.layout.width = "90%"
prev_btn = Button(description="Preview")
next_btn = Button(description="Next")
loop_btn = Button(description="AutoPlay")
loop_btn.layout.width = "200px"
imgbox = Image(format='jpg')


def read_dir(_=None):
    global ROOT, idx, pics
    idx = 0
    ROOT = pos_txt.value
    try:
        pics = os.listdir(ROOT)
    except Exception as e:
        pics = []

    drpbox.options = [(name, i) for i, name in enumerate(pics)]
    show_pic()


def show_pic():
    global idx, pics
    prev_btn.disabled = next_btn.disabled = False
    if idx <= 0:
        idx = 0
        prev_btn.disabled = True
    if idx >= len(pics) - 1:
        idx = len(pics) - 1
        next_btn.disabled = True
    drpbox.description = "[{}/{}]:".format(idx + 1, len(pics))
    if len(pics) == 0:
        return None
    drpbox.value = idx
    crtpath = os.path.join(pos_txt.value, drpbox.label)
    _, ext = os.path.splitext(crtpath)
    if ext in IMAGE_FMT:
        img = cv2.imread(crtpath)
        imgbox.value = cv2.imencode('.jpg', img)[1].tobytes()
    elif ext in VIDEO_FMT:
        cap = cv2.VideoCapture(crtpath)
        fps = cap.get(cv2.CAP_PROP_FPS)
        while (cap.isOpened()):
            ret, frame = cap.read()
            if not ret:
                break
            imgbox.value = cv2.imencode('.jpg', frame)[1].tobytes()
            sleep(1 / fps)
    return crtpath


def choose_pic(attval):
    global idx
    idx = attval['new']
    show_pic()


def prev_pic(sender):
    global idx
    idx -= 1
    show_pic()


def next_pic(sender):
    global idx
    idx += 1
    show_pic()


def loop_pic(sender):
    if loop_btn.description == "AutoPlay":
        loop_btn.description = "Stop loop by Kernel->Interrupt"
        loop_btn.disabled = True
        global idx
        try:
            while idx <= len(pics) - 1:
                show_pic()
                sleep(0.2)
                idx += 1
        except KeyboardInterrupt:
            print("STOP")
            pass
    loop_btn.description = "AutoPlay"
    loop_btn.disabled = False


pos_txt.on_submit(read_dir)
jmp_btn.on_click(read_dir)
drpbox.observe(choose_pic, names='value')
prev_btn.on_click(prev_pic)
next_btn.on_click(next_pic)
loop_btn.on_click(loop_pic)
display(info_lbl, HBox([pos_txt, jmp_btn]), drpbox,
        HBox([prev_btn, next_btn, loop_btn]), imgbox)

# init
pos_txt.value = ROOT
read_dir()

Label(value='')

Dropdown(description='[0/0]:', layout=Layout(width='90%'), options=(), value=None)

Image(value=b'', format='jpg')